In [1]:
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor

import re
import sys, os, glob
import json
from collections import OrderedDict

sys.path.append('..')
# import fidle.pwk    as pwk
import fidle.config as config
                

In [2]:
directories_to_index = ['LinearReg', 'IRIS', 'BHPD', 'MNIST', 'GTSRB', 'IMDB', 'SYNOP', 'VAE', 'Misc']

In [3]:

def get_notebooks(directories, top_dir='..'):
    '''
    Return a list of notebooks from a given list of directories
    args:
        directories : list of directories
        top_dir : location of theses directories
    return:
        notebooks : notebooks filename list (without top_dir prefix)
    '''
    notebooks = []
    
    for d in directories:
        filenames = glob.glob( f'{top_dir}/{d}/*.ipynb')
        filenames.sort()
        notebooks.extend(filenames)

    notebooks = [ x.replace(f'{top_dir}/','') for x in notebooks]
    return notebooks


def get_infos(filename, top_dir='..'):
    '''
    Extract informations from a fidle notebook.
    Informations are dirname, basename, id, title, description and are extracted from comments tags in markdown.
    args:
        filename : Notebook filename
    return:
        dict : with infos.
    '''

    about={}
    about['dirname']     = os.path.dirname(filename)
    about['basename']    = os.path.basename(filename)
    about['id']          = '??'
    about['title']       = '??'
    about['description'] = '??'
    
    # ---- Read notebook
    #
    notebook = nbformat.read(f'{top_dir}/{filename}', nbformat.NO_CONVERT)
    
    # ---- Get id, title and desc tags
    #
    for cell in notebook.cells:

        if cell['cell_type'] == 'markdown':

            find = re.findall(r'<\!-- TITLE -->\s*\[(.*)\]\s*-\s*(.*)\n',cell.source)
            if find:
                about['id']    = find[0][0]
                about['title'] = find[0][1]

            find = re.findall(r'<\!-- DESC -->\s*(.*)\n',cell.source)
            if find:
                about['description']  = find[0]

    return about
                

def get_catalog(notebooks_list, top_dir='..'):
    '''
    Return an OrderedDict of notebooks attributes.
    Keys are notebooks id.
    args:
        notebooks_list : list of notebooks filenames
        top_dir : Location of theses notebooks
    return:
        OrderedDict : {<notebook id> : { description} }
    '''
    
    catalog = OrderedDict()

    for nb in notebooks_list:
        about = get_infos(nb, top_dir='..')
        id=about['id']
        catalog[id] = about

    return catalog
        
        

In [4]:

# ---- Get the notebook list
#
notebooks_list = get_notebooks(directories_to_index)

# ---- Get a detailled catalog for this list
#
catalog = get_catalog(notebooks_list)

with open(config.CATALOG_FILE,'wt') as fp:
        json.dump(catalog,fp,indent=4)

# ---- Create a markdown index
#
lines=['| | |','|--|--|']
tab='&nbsp;'*5
for id, about in catalog.items():
    id          = about['id']
    dirname     = about['dirname']
    basename    = about['basename']
    title       = about['title']
    description = about['description']
      
#     lines.append( f'[[{id}] - {title}]({dirname}/{basename})  ' )
#     lines.append( f'{tab}{description}  ')
    lines.append( f'|{id}| [{title}]({dirname}/{basename})<br>{description}|')

index = '\n'.join(lines)
    
# ---- Load README.md
#
with open('../README.md','r') as fp:
    readme=fp.read()
    
# ---- Update index
#
debut = '<!-- INDEX_BEGIN -->'
fin   = '<!-- INDEX_END -->'

readme = re.sub(f'{debut}.*{fin}',f'{debut}\n{index}\n{fin}',readme, flags=re.DOTALL)

# ---- Update version
#
debut = '<!-- VERSION_BEGIN -->'
fin   = '<!-- VERSION_END -->'

readme = re.sub(f'{debut}.*{fin}',f'{debut}\n{config.VERSION}\n{fin}',readme, flags=re.DOTALL)

# ---- Save it
#
with open('../README.md','wt') as fp:
    fp.write(readme)

print('README.md is updated.')

README.md is updated.


In [5]:
# ---- Create Notebook
#
notebook = nbformat.v4.new_notebook()
new_cell = nbformat.v4.new_markdown_cell(source=readme)
notebook.cells.append(new_cell)

# ---- Run it
#
ep = ExecutePreprocessor(timeout=600, kernel_name="python3")
ep.preprocess(notebook)

# ---- Save it
#
with open('../test.ipynb', mode="w", encoding='utf-8') as fp:
    nbformat.write(notebook, fp)